# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import torch
from torch import nn
from typing import List, Set, Dict
from collections import defaultdict
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
import string
from dataclasses import dataclass, field
from typing import List, Set

@dataclass
class GameState:
    """
    Represents the state of a Hangman game. Includes properties to dynamically
    calculate available and guessed letters to ensure compatibility with various agents.
    """
    word_pattern: List[str]
    lives_remaining: int
    correct_guesses: Set[str] = field(default_factory=set)
    incorrect_guesses: Set[str] = field(default_factory=set)

    @property
    def guessed_letters(self) -> Set[str]:
        """
        A property that returns the set of all letters guessed so far,
        both correct and incorrect. This fixes the new AttributeError.
        """
        return self.correct_guesses.union(self.incorrect_guesses)

    @property
    def available_letters(self) -> Set[str]:
        """
        A property that returns the set of letters in the alphabet that have not
        been guessed yet.
        """
        all_letters = set(string.ascii_lowercase)
        # We can now use our new 'guessed_letters' property here for cleaner code!
        return all_letters - self.guessed_letters

In [ ]:
class CharacterBERT(nn.Module):
    """The custom Transformer model for Hangman."""
    def __init__(self, vocab_size, max_len, hidden_size, nhead, num_layers, game_state_size):
        super().__init__()
        self.char_embedding = nn.Embedding(vocab_size, hidden_size)
        self.position_embedding = nn.Embedding(max_len, hidden_size)
        self.game_state_encoder = nn.Linear(game_state_size, hidden_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.mlm_head = nn.Linear(hidden_size, vocab_size)
        self.register_buffer('positions', torch.arange(max_len))

    def forward(self, input_ids, game_state_vector):
        seq_len = input_ids.size(1)
        x = self.char_embedding(input_ids) + self.position_embedding(self.positions[:seq_len])
        x = x + self.game_state_encoder(game_state_vector).unsqueeze(1)
        logits = self.mlm_head(self.transformer_encoder(x))
        return logits

In [ ]:
class BERTAgent:
    def __init__(self, model_path: str, wordlist_path: str, device: str):
        self.device = device
        self.model = CharacterBERT(VOCAB_SIZE, MAX_LEN, HIDDEN_SIZE, NHEAD, NUM_LAYERS, GAME_STATE_SIZE)
        self.model.load_state_dict(torch.load(model_path, map_location=device))
        self.model.to(device)
        self.model.eval()
        self.linguistic_filter = LengthAwareFilter(wordlist_path)
        self.alphabet = string.ascii_lowercase

    def _get_char_energies(self, game_state: GameState, candidates: List[str]) -> Dict[str, float]:
        input_pattern = [VOCAB[c] if c != '_' else VOCAB['[MASK]'] for c in game_state.word_pattern]
        input_ids = torch.tensor(input_pattern + [VOCAB['[PAD]']] * (MAX_LEN - len(input_pattern)), dtype=torch.long).unsqueeze(0).to(self.device)
        incorrect_mask = [1.0 if c in game_state.incorrect_guesses else 0.0 for c in self.alphabet]
        game_state_vector = torch.tensor([game_state.lives_remaining / 6.0] + incorrect_mask, dtype=torch.float32).unsqueeze(0).to(self.device)
        with torch.no_grad():
            logits = self.model(input_ids, game_state_vector)
        mask_indices = [i for i, c in enumerate(game_state.word_pattern) if c == '_']
        if not mask_indices:
            return {}
        masked_logits = logits[0, mask_indices, :]
        probabilities = torch.softmax(masked_logits, dim=-1)
        energies = {c: -probabilities[:, VOCAB[c]].sum().item() for c in candidates}
        return energies

    def get_ranked_guesses(self, game_state: GameState) -> List[str]:
        if not game_state.available_letters:
            return []
        candidates = self.linguistic_filter.get_candidates(game_state, top_n=len(game_state.available_letters))
        energies = self._get_char_energies(game_state, candidates)
        if not energies:
            return candidates
        return sorted(candidates, key=lambda c: energies.get(c, 0))

    def choose_character(self, game_state: GameState) -> str:
        guesses = self.get_ranked_guesses(game_state)
        return guesses[0] if guesses else ''

In [ ]:
class LengthAwareFilter:
    def __init__(self, wordlist_path: str):
        self.frequencies_by_length = defaultdict(lambda: defaultdict(int))
        self.fallback_frequencies = {
            'e': 12.7, 't': 9.1, 'a': 8.2, 'o': 7.5, 'i': 7.0, 'n': 6.7,
            's': 6.3, 'h': 6.1, 'r': 6.0, 'd': 4.3, 'l': 4.0, 'c': 2.8,
            'u': 2.8, 'm': 2.4, 'w': 2.3, 'f': 2.2, 'g': 2.0, 'y': 2.0,
            'p': 1.9, 'b': 1.5, 'v': 1.0, 'k': 0.8, 'j': 0.2, 'x': 0.2,
            'q': 0.1, 'z': 0.1
        }

        try:
            with open(wordlist_path, 'r') as f:
                words = [line.strip().lower() for line in f if line.strip().isalpha()]
            for word in words:
                length = len(word)
                for char in set(word):
                    self.frequencies_by_length[length][char] += 1
        except FileNotFoundError:
            print(f"WARNING: Wordlist not found at '{wordlist_path}'. Using fallback frequencies only.")

    def get_candidates(self, game_state: GameState, top_n: int = 10) -> List[str]:
        available_letters = game_state.available_letters
        word_len = len(game_state.word_pattern)
        freq_map = self.frequencies_by_length.get(word_len, self.fallback_frequencies)
        sorted_letters = sorted(available_letters, key=lambda c: freq_map.get(c, 0), reverse=True)
        return sorted_letters[:top_n]

In [ ]:
# import re
# from collections import defaultdict, Counter
# from typing import List, Set, Dict
# import math

# class LengthAwareFilter:
#     def __init__(self, wordlist_path: str):
#         self.word_dict = defaultdict(list)  # length -> [words]
#         self.frequencies_by_length = defaultdict(lambda: defaultdict(int))
#         self.positional_frequencies = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
#         self.ngram_frequencies = defaultdict(lambda: defaultdict(int))

#         # Fallback frequencies (your existing approach)
#         self.fallback_frequencies = {
#             'e': 12.7, 't': 9.1, 'a': 8.2, 'o': 7.5, 'i': 7.0, 'n': 6.7,
#             's': 6.3, 'h': 6.1, 'r': 6.0, 'd': 4.3, 'l': 4.0, 'c': 2.8,
#             'u': 2.8, 'm': 2.4, 'w': 2.2, 'g': 2.0, 'y': 2.0,
#             'p': 1.9, 'b': 1.5, 'v': 1.0, 'k': 0.8, 'j': 0.2, 'x': 0.2,
#             'q': 0.1, 'z': 0.1
#         }

#         try:
#             self._load_and_analyze_wordlist(wordlist_path)
#         except FileNotFoundError:
#             print(f"WARNING: Wordlist not found at '{wordlist_path}'. Using fallback frequencies only.")

#     def _load_and_analyze_wordlist(self, wordlist_path: str):
#         """Load words and build comprehensive frequency maps"""
#         with open(wordlist_path, 'r') as f:
#             words = [line.strip().lower() for line in f if line.strip().isalpha()]

#         for word in words:
#             length = len(word)
#             self.word_dict[length].append(word)

#             # Overall frequency by length
#             for char in set(word):
#                 self.frequencies_by_length[length][char] += 1

#             # Positional frequency analysis
#             for pos, char in enumerate(word):
#                 self.positional_frequencies[length][pos][char] += 1

#             # Build n-grams (bigrams and trigrams)
#             for i in range(len(word) - 1):
#                 bigram = word[i:i+2]
#                 self.ngram_frequencies[f"2_{length}"][bigram] += 1

#             for i in range(len(word) - 2):
#                 trigram = word[i:i+3]
#                 self.ngram_frequencies[f"3_{length}"][trigram] += 1

#     def filter_possible_words(self, game_state) -> List[str]:
#         """Filter words that match the current pattern and constraints"""
#         word_len = len(game_state.word_pattern)
#         possible_words = []

#         # Get words of correct length
#         candidates = self.word_dict.get(word_len, [])

#         # Build regex pattern from current state
#         pattern = self._build_regex_pattern(game_state)
#         regex = re.compile(pattern)

#         for word in candidates:
#             # Must match revealed pattern
#             if not regex.match(word):
#                 continue

#             # Must not contain any incorrect guesses
#             if any(letter in word for letter in game_state.incorrect_guesses):
#                 continue

#             # Must contain all correct guesses
#             if not all(letter in word for letter in game_state.correct_guesses):
#                 continue

#             possible_words.append(word)

#         return possible_words

#     def _build_regex_pattern(self, game_state) -> str:
#         """Convert word pattern to regex"""
#         pattern = ""
#         for char in game_state.word_pattern:
#             if char == '_':
#                 # Exclude incorrect guesses
#                 excluded = ''.join(game_state.incorrect_guesses)
#                 if excluded:
#                     pattern += f"[^{excluded}]"
#                 else:
#                     pattern += "."
#             else:
#                 pattern += char
#         return pattern

#     def get_candidates_entropy_based(self, game_state, top_n: int = 10) -> List[str]:
#         """Use information theory to find letters that eliminate most possibilities"""
#         possible_words = self.filter_possible_words(game_state)

#         if not possible_words:
#             return self._fallback_guess(game_state, top_n)

#         letter_scores = {}
#         available_letters = game_state.available_letters

#         for letter in available_letters:
#             # Calculate entropy reduction for this letter
#             with_letter = sum(1 for word in possible_words if letter in word)
#             without_letter = len(possible_words) - with_letter

#             if with_letter == 0 or without_letter == 0:
#                 # No information gain
#                 letter_scores[letter] = 0
#             else:
#                 # Calculate entropy reduction
#                 total = len(possible_words)
#                 p_with = with_letter / total
#                 p_without = without_letter / total

#                 entropy_reduction = -(p_with * math.log2(p_with) + p_without * math.log2(p_without))
#                 letter_scores[letter] = entropy_reduction

#         # Sort by entropy reduction (higher is better)
#         sorted_letters = sorted(letter_scores.keys(), key=lambda x: letter_scores[x], reverse=True)
#         return sorted_letters[:top_n]

#     def get_candidates_frequency_based(self, game_state, top_n: int = 10) -> List[str]:
#         """Improved frequency-based approach using filtered word list"""
#         possible_words = self.filter_possible_words(game_state)

#         if not possible_words:
#             return self._fallback_guess(game_state, top_n)

#         # Count frequency of each letter in possible words
#         letter_counts = Counter()
#         for word in possible_words:
#             for letter in set(word):  # Count each letter once per word
#                 if letter in game_state.available_letters:
#                     letter_counts[letter] += 1

#         # Sort by frequency
#         sorted_letters = [letter for letter, _ in letter_counts.most_common(top_n)]
#         return sorted_letters

#     def get_candidates_positional(self, game_state, top_n: int = 10) -> List[str]:
#         """Use positional frequency analysis"""
#         possible_words = self.filter_possible_words(game_state)

#         if not possible_words:
#             return self._fallback_guess(game_state, top_n)

#         word_len = len(game_state.word_pattern)
#         letter_scores = defaultdict(float)

#         # Score letters based on positional probability
#         for pos, char in enumerate(game_state.word_pattern):
#             if char == '_':  # Unknown position
#                 pos_freq = self.positional_frequencies[word_len][pos]
#                 total_at_pos = sum(pos_freq.values())

#                 if total_at_pos > 0:
#                     for letter in game_state.available_letters:
#                         probability = pos_freq[letter] / total_at_pos
#                         letter_scores[letter] += probability

#         # Sort by combined positional score
#         sorted_letters = sorted(letter_scores.keys(),
#                               key=lambda x: letter_scores[x], reverse=True)
#         return sorted_letters[:top_n]

#     def get_candidates_ngram_based(self, game_state, top_n: int = 10) -> List[str]:
#         """Use n-gram analysis for context-aware guessing"""
#         possible_words = self.filter_possible_words(game_state)

#         if not possible_words:
#             return self._fallback_guess(game_state, top_n)

#         word_len = len(game_state.word_pattern)
#         letter_scores = defaultdict(float)

#         # Analyze bigrams and trigrams around unknown positions
#         for pos, char in enumerate(game_state.word_pattern):
#             if char == '_':
#                 # Check preceding context
#                 if pos > 0 and game_state.word_pattern[pos-1] != '_':
#                     prev_char = game_state.word_pattern[pos-1]
#                     bigram_key = f"2_{word_len}"

#                     for letter in game_state.available_letters:
#                         bigram = prev_char + letter
#                         count = self.ngram_frequencies[bigram_key].get(bigram, 0)
#                         letter_scores[letter] += count

#                 # Check following context
#                 if pos < len(game_state.word_pattern) - 1 and game_state.word_pattern[pos+1] != '_':
#                     next_char = game_state.word_pattern[pos+1]
#                     bigram_key = f"2_{word_len}"

#                     for letter in game_state.available_letters:
#                         bigram = letter + next_char
#                         count = self.ngram_frequencies[bigram_key].get(bigram, 0)
#                         letter_scores[letter] += count

#         sorted_letters = sorted(letter_scores.keys(),
#                               key=lambda x: letter_scores[x], reverse=True)
#         return sorted_letters[:top_n]

#     def get_candidates_combined(self, game_state, top_n: int = 10) -> List[str]:
#         """Combine multiple strategies for best results"""
#         # Get candidates from different methods
#         entropy_candidates = self.get_candidates_entropy_based(game_state, top_n * 2)
#         frequency_candidates = self.get_candidates_frequency_based(game_state, top_n * 2)
#         positional_candidates = self.get_candidates_positional(game_state, top_n * 2)
#         ngram_candidates = self.get_candidates_ngram_based(game_state, top_n * 2)

#         # Score each letter based on ranking in each method
#         letter_scores = defaultdict(float)
#         methods = [
#             (entropy_candidates, 0.3),      # High weight for entropy
#             (frequency_candidates, 0.25),   # Medium-high for frequency
#             (positional_candidates, 0.25),  # Medium-high for position
#             (ngram_candidates, 0.2)         # Medium for n-grams
#         ]

#         for candidates, weight in methods:
#             for i, letter in enumerate(candidates):
#                 # Higher score for higher ranking (lower index)
#                 score = (len(candidates) - i) * weight
#                 letter_scores[letter] += score

#         # Sort by combined score
#         sorted_letters = sorted(letter_scores.keys(),
#                               key=lambda x: letter_scores[x], reverse=True)
#         return sorted_letters[:top_n]

#     def _fallback_guess(self, game_state, top_n: int) -> List[str]:
#         """Fallback to length-based frequency when no words match"""
#         word_len = len(game_state.word_pattern)
#         freq_map = self.frequencies_by_length.get(word_len, self.fallback_frequencies)

#         sorted_letters = sorted(game_state.available_letters,
#                               key=lambda c: freq_map.get(c, 0), reverse=True)
#         return sorted_letters[:top_n]

#     # Main interface method - use the best combined approach
#     def get_candidates(self, game_state, top_n: int = 10) -> List[str]:
#         """Main method - uses combined approach for best results"""
#         return self.get_candidates_combined(game_state, top_n)


In [ ]:
class FilteringAgent:
    def __init__(self, wordlist_path: str):
        try:
            with open(wordlist_path, "r") as f:
                self.full_dictionary = f.read().splitlines()
        except FileNotFoundError:
            print(f"FATAL: Wordlist not found at {wordlist_path}")
            self.full_dictionary = ['apple', 'banana', 'hangman']

        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.letters_absent = []

    def reset(self):
        self.current_dictionary = self.full_dictionary.copy()
        self.letters_absent = []

    def choose_character(self, game_state: GameState) -> str:
        clean_word = "".join(game_state.word_pattern).replace("_", ".")
        guessed_letters = game_state.guessed_letters
        len_word = len(clean_word)
        self.letters_absent = list(game_state.incorrect_guesses)

        filtered = []
        for word in self.current_dictionary:
            if len(word) != len_word:
                continue
            if re.fullmatch(clean_word, word) and not any(ch in word for ch in self.letters_absent):
                filtered.append(word)

        if filtered:
            self.current_dictionary = filtered

        if not self.current_dictionary:
            for ch, _ in self.full_dictionary_common_letter_sorted:
                if ch not in guessed_letters:
                    return ch

        if len(self.current_dictionary) == 1:
            for ch in self.current_dictionary[0]:
                if ch not in guessed_letters:
                    return ch

        counter = collections.Counter("".join(self.current_dictionary))
        for ch, _ in counter.most_common():
            if ch not in guessed_letters:
                return ch

        if game_state.available_letters:
            return game_state.available_letters[0]
        return 'a'


In [ ]:
VOCAB = {'[PAD]': 0, '[MASK]': 1, **{char: i+2 for i, char in enumerate(string.ascii_lowercase)}}
VOCAB_SIZE = len(VOCAB)
MAX_LEN = 35
HIDDEN_SIZE = 128
NHEAD = 4
NUM_LAYERS = 4
GAME_STATE_SIZE = 27  # 1 for lives_remaining + 26 for one-hot incorrect guesses
BATCH_SIZE = 64
EPOCHS = 3  # Increase for better performance with a larger wordlist
LEARNING_RATE = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import os
import random
import collections
from dataclasses import dataclass, field
from typing import List, Set, Tuple
from transformers import pipeline, logging
import sys
from tqdm import tqdm

# --- Configuration ---
DEVICE = "cuda" if "torch" in sys.modules and sys.modules["torch"].cuda.is_available() else "cpu"
MODEL_SAVE_PATH = "/content/drive/MyDrive/HMan/character_bert_hangman_final.pth"
WORDLIST_PATH = "/content/drive/MyDrive/HMan/words.txt"
NUM_GAMES = 1000
MAX_LIVES = 6

# --- Setup: Ensure wordlist exists ---
if not os.path.exists(WORDLIST_PATH):
    print(f"Error: Wordlist not found at '{WORDLIST_PATH}'.")
    print("Creating a dummy 'words.txt' for demonstration purposes.")
    dummy_words = [
        "apple", "banana", "python", "hangman", "challenge", "computer", "language",
        "entropy", "strategy", "prediction", "failure", "character", "wizard", "galaxy",
        "quantum", "zodiac", "jigsaw", "kayak", "oxygen", "pixel", "rhythm"
    ]
    with open(WORDLIST_PATH, "w") as f:
        f.write("\n".join(dummy_words))

# Suppress verbose loading messages from transformers
logging.set_verbosity_error()
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

class HybridFinalGuessAgent:
    def __init__(self, model_path: str, wordlist_path: str, device: str):
        self.device = device
        self.model = CharacterBERT(VOCAB_SIZE, MAX_LEN, HIDDEN_SIZE, NHEAD, NUM_LAYERS, GAME_STATE_SIZE)
        self.model.load_state_dict(torch.load(model_path, map_location=device))
        self.model.to(device)
        self.model.eval()
        self.linguistic_filter = LengthAwareFilter(wordlist_path)
        self.alphabet = string.ascii_lowercase

    def _get_char_energies(self, game_state: GameState, candidates: List[str]) -> Dict[str, float]:
        input_pattern = [VOCAB[c] if c != '_' else VOCAB['[MASK]'] for c in game_state.word_pattern]
        input_ids = torch.tensor(
            input_pattern + [VOCAB['[PAD]']] * (MAX_LEN - len(input_pattern)),
            dtype=torch.long
        ).unsqueeze(0).to(self.device)

        incorrect_mask = [1.0 if c in game_state.incorrect_guesses else 0.0 for c in self.alphabet]
        game_state_vector = torch.tensor(
            [game_state.lives_remaining / 6.0] + incorrect_mask,
            dtype=torch.float32
        ).unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits = self.model(input_ids, game_state_vector)

        mask_indices = [i for i, c in enumerate(game_state.word_pattern) if c == '_']
        if not mask_indices:
            return {}

        masked_logits = logits[0, mask_indices, :]
        probabilities = torch.softmax(masked_logits, dim=-1)
        energies = {c: -probabilities[:, VOCAB[c]].sum().item() for c in candidates}
        return energies

    def get_ranked_guesses(self, game_state: GameState) -> List[str]:
        if not game_state.available_letters:
            return []

        candidates = self.linguistic_filter.get_candidates(game_state, top_n=len(game_state.available_letters))
        energies = self._get_char_energies(game_state, candidates)

        if not energies:
            return candidates

        return sorted(candidates, key=lambda c: energies.get(c, 0))

    def choose_character(self, game_state: GameState) -> str:
        guesses = self.get_ranked_guesses(game_state)
        return guesses[0] if guesses else ''



# # --- The Main Agent Controller ---
# class TriModeAgent:
#     def __init__(self, bert_model_path: str, wordlist_path: str, device: str):
#         self.bert_agent = BERTAgent(bert_model_path, wordlist_path, device)
#         self.entropy_agent = EntropyAgent(wordlist_path)
#         self.hybrid_agent = HybridFinalGuessAgent("/content/drive/My Drive/HMan/cbertlast.pth", wordlist_path, device)
#     def choose_character(self, game_state: GameState, verbose=False) -> str:
#         word_pattern = game_state.word_pattern
#         word_len = len(word_pattern)
#         num_blanks = word_pattern.count('_')
#         if verbose:
#             print(f"  [State] Pattern: {''.join(word_pattern)}, Lives: {game_state.lives_remaining}, Blanks: {num_blanks}, Len: {word_len}")
#         if 1 <= num_blanks <= 2 and 3 <= word_len <= 7:
#             print("  [Logic] Using HybridFinalGuessAgent (Endgame Strategy)")
#             if verbose: print("  [Logic] Using HybridFinalGuessAgent (Endgame Strategy)")
#             return self.bert_agent.choose_character(game_state)
#             # self.entropy_agent2.choose_character(game_state)
#         elif 3 <= word_len <= 7:
#             print("  [Logic] Using EntropyAgent (Short Word Strategy)")
#             if verbose: print("  [Logic] Using EntropyAgent (Short Word Strategy)")
#             return self.entropy_agent.choose_character(game_state)
#         else:
#             print("  [Logic] Using BERTAgent (General Strategy)")
#             if verbose: print("  [Logic] Using BERTAgent (General Strategy)")
#             return self.hybrid_agent.choose_character(game_state)

# class TriModeAgent:
#     def __init__(self, bert_model_path: str, wordlist_path: str, device: str):
#         self.bert_agent = BERTAgent(bert_model_path, wordlist_path, device)
#         self.entropy_agent = EntropyAgent(wordlist_path)
#         self.hybrid_agent = HybridFinalGuessAgent("/content/drive/My Drive/HMan/cbertlast.pth", wordlist_path, device)

#         # Hardcoded optimal calling order from the image for lengths 1–4
#         self.optimal_calls = {
#             1: list("AI"),
#             2: list("AOEIU"),
#             3: list("ETNADYG"),
#             4: list("EATNYLD")
#         }
#         self.guess_history = set()

#     def choose_character(self, game_state: GameState, verbose=False) -> str:
#         word_pattern = game_state.word_pattern
#         word_len = len(word_pattern)
#         num_blanks = word_pattern.count('_')
#         guessed = set(game_state.guessed_letters)

#         if verbose:
#             print(f"  [State] Pattern: {''.join(word_pattern)}, Lives: {game_state.lives_remaining}, "
#                   f"Blanks: {num_blanks}, Len: {word_len}, Guessed: {guessed}")

#         # Use lookup strategy for word lengths 1–4
#         if word_len in self.optimal_calls and num_blanks == 1:
#             print("  [Logic] Using Lookup Strategy (Optimal Calling Order)")
#             for ch in self.optimal_calls[word_len]:
#                 if ch not in guessed:
#                     return ch

#         # Use HybridFinalGuessAgent for endgame
#         if 1 <= num_blanks <= 2 and 3<=word_len <=7:
#             print("  [Logic] Using HybridFinalGuessAgent (Endgame Strategy)")
#             # return self.bert_agent.choose_character(game_state)
#             return self.hybrid_agent.choose_character(game_state)

#         # Use EntropyAgent for short words
#         elif 3 <= word_len <= 7:
#             print("  [Logic] Using EntropyAgent (Short Word Strategy)")
#             return self.entropy_agent.choose_character(game_state)

#         # Use BERTAgent otherwise
#         else:
#             print("  [Logic] Using BERTAgent (General Strategy)")
#             return self.hybrid_agent.choose_character(game_state)


# class TriModeAgent:
#     def __init__(self, bert_model_path: str, wordlist_path: str, device: str):
#         self.bert_agent = BERTAgent(bert_model_path, wordlist_path, device)
#         self.entropy_agent = EntropyAgent(wordlist_path)
#         self.hybrid_agent = HybridFinalGuessAgent("/content/drive/My Drive/HMan/cbertlast.pth", wordlist_path, device)

#         self.optimal_calls = {
#             1: list("ai"),
#             2: list("aoeiu"),
#             3: list("etnadyg"),
#             4: list("eatnyld"),
#             5: list("eyatrnl")  # Add if needed
#         }

#         self.guess_history = set()
#         self.hybrid_mistakes = 0
#         self.previous_lives = None

#     def choose_character(self, game_state: GameState, verbose=False) -> str:
#         word_pattern = game_state.word_pattern
#         word_len = len(word_pattern)
#         num_blanks = word_pattern.count('_')
#         guessed = set(game_state.guessed_letters)

#         # Track mistakes by hybrid agent
#         if self.previous_lives is not None and game_state.lives_remaining < self.previous_lives:
#             self.hybrid_mistakes += 1
#         self.previous_lives = game_state.lives_remaining

#         if verbose:
#             print(f"  [State] Pattern: {''.join(word_pattern)}, Lives: {game_state.lives_remaining}, "
#                   f"Blanks: {num_blanks}, Len: {word_len}, Guessed: {guessed}, HybridMistakes: {self.hybrid_mistakes}")

#         # Use lookup strategy for word lengths 1–4 if 1 blank
#         # if word_len in self.optimal_calls and num_blanks == 1:
#         #     print("  [Logic] Using Lookup Strategy (Optimal Calling Order)")
#         #     for ch in self.optimal_calls[word_len]:
#         #         if ch not in guessed:
#         #             return ch

#         # Fallback to optimal_calls if hybrid agent made 2+ mistakes in endgame
#         if 2 <= word_len <= 5 and 1 <= num_blanks <= 2 and self.hybrid_mistakes >= 1:
#             print("  [Logic] Using Lookup Strategy (Fallback due to Hybrid Mistakes)")
#             for ch in self.optimal_calls.get(word_len, []):
#                 if ch not in guessed:
#                     return ch

#         # Use HybridFinalGuessAgent for endgame
#         if 1 <= num_blanks <= 2 and 3 <= word_len <= 7:
#             print("  [Logic] Using HybridFinalGuessAgent (Endgame Strategy)")
#             return self.hybrid_agent.choose_character(game_state)

#         # Use EntropyAgent for short words
#         elif 3 <= word_len <= 7:
#             print("  [Logic] Using EntropyAgent (Short Word Strategy)")
#             return self.entropy_agent.choose_character(game_state)

#         # Use BERTAgent otherwise
#         else:
#             print("  [Logic] Using BERTAgent (General Strategy)")
#             return self.hybrid_agent.choose_character(game_state)

#     def reset(self):
#         """Call this after each game ends."""
#         self.hybrid_mistakes = 0
#         self.previous_lives = None
#         self.guess_history.clear()


class TriModeAgent:
    def __init__(self, bert_model_path: str, wordlist_path: str, device: str):
        self.bert_agent = BERTAgent(bert_model_path, wordlist_path, device)
        self.entropy_agent = FilteringAgent(wordlist_path)
        self.hybrid_agent = HybridFinalGuessAgent("/content/drive/My Drive/HMan/cbertlast.pth", wordlist_path, device)

        self.optimal_calls = {
            1: list("ai"),
            2: list("aoeiu"),
            3: list("etnadyg"),
            4: list("eatnyld"),
            5: list("eyatrnl"),
        }

        self.guess_history = set()
        self.hybrid_mistakes = 0
        self.previous_lives = None
        self.last_used_agent = None  # Track which agent was used last

    def choose_character(self, game_state: GameState, verbose=False) -> str:
        word_pattern = game_state.word_pattern
        word_len = len(word_pattern)
        num_blanks = word_pattern.count('_')
        guessed = set(game_state.guessed_letters)

        # Update hybrid mistake only if hybrid agent was used last and lives decreased
        if self.previous_lives is not None and game_state.lives_remaining < self.previous_lives:
            if self.last_used_agent == "hybrid":
                self.hybrid_mistakes += 1
        self.previous_lives = game_state.lives_remaining

        if verbose:
            print(f"  [State] Pattern: {''.join(word_pattern)}, Lives: {game_state.lives_remaining}, "
                  f"Blanks: {num_blanks}, Len: {word_len}, Guessed: {guessed}, HybridMistakes: {self.hybrid_mistakes}")

        # Use lookup fallback if hybrid made 1+ mistakes in critical stage
        if 2 <= word_len <= 5 and 1 <= num_blanks <= 2 and self.hybrid_mistakes >= 1:
            print("  [Logic] Using Lookup Strategy (Fallback due to Hybrid Mistakes)")
            self.last_used_agent = "lookup"
            for ch in self.optimal_calls.get(word_len, []):
                if ch not in guessed:
                    return ch

        # Use HybridFinalGuessAgent for endgame
        if 1 <= num_blanks <= 2 and 3<=word_len<=7:
            print("  [Logic] Using HybridFinalGuessAgent (Endgame Strategy)")
            self.last_used_agent = "hybrid"
            return self.hybrid_agent.choose_character(game_state)

        # Use EntropyAgent for short words
        elif 3 <= word_len <= 7:
            print("  [Logic] Using EntropyAgent (Short Word Strategy)")
            self.last_used_agent = "entropy"
            return self.entropy_agent.choose_character(game_state)

        # Use BERTAgent otherwise
        else:
            print("  [Logic] Using BERTAgent (General Strategy)")
            self.last_used_agent = "bert"
            return self.bert_agent.choose_character(game_state)

    def reset(self):
        """Call this after each game ends."""
        self.hybrid_mistakes = 0
        self.previous_lives = None
        self.guess_history.clear()
        self.last_used_agent = None




In [ ]:

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "/content/drive/MyDrive/HMan/words.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):  # word example: "_ p p _ e "
        pattern = word.replace(" ", "")

        # Compute lives remaining from number of incorrect guesses
        incorrect_guesses = [ch for ch in self.guessed_letters if ch not in pattern]
        correct_guesses = [ch for ch in self.guessed_letters if ch in pattern]
        lives_remaining = MAX_LIVES - len(incorrect_guesses)

        # Prepare game state
        game_state = GameState(
            word_pattern=list(pattern),
            lives_remaining=lives_remaining,
            correct_guesses=set(correct_guesses),
            incorrect_guesses=set(incorrect_guesses)
        )

        # Initialize agent if not already done
        if not hasattr(self, 'agent'):
            self.agent = TriModeAgent(
                bert_model_path=MODEL_SAVE_PATH,
                wordlist_path=WORDLIST_PATH,
                device=DEVICE
            )
            # self.agent = TriModeAgent(
            #     bert_model_path=MODEL_SAVE_PATH,
            #     wordlist_path=WORDLIST_PATH,
            #     device="cpu",
            #     rl_model_path="/content/drive/MyDrive/HMan/hangman_q_table.pkl"
            # )

        # Detect game over and reset agent
        if "_" not in pattern or lives_remaining <= 0:
            self.agent.reset()
            return ''

        return self.agent.choose_character(game_state, verbose=False)


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [ ]:
api = HangmanAPI(access_token="api", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 243bd3df46f2. # of tries remaining: 6. Word: _ _ _ _ _ .
  [Logic] Using EntropyAgent (Short Word Strategy)
Guessing letter: e
Sever response: {'game_id': '243bd3df46f2', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ '}
  [Logic] Using EntropyAgent (Short Word Strategy)
Guessing letter: i
Sever response: {'game_id': '243bd3df46f2', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ '}
  [Logic] Using EntropyAgent (Short Word Strategy)
Guessing letter: a
Sever response: {'game_id': '243bd3df46f2', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a _ a '}
  [Logic] Using EntropyAgent (Short Word Strategy)
Guessing letter: n
Sever response: {'game_id': '243bd3df46f2', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ a n a '}
  [Logic] Using HybridFinalGuessAgent (Endgame Strategy)
Guessing letter: r
Sever response: {'game_id': '243bd3df46f2', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ a n a '}
  [Logic] Us

In [ ]:
for i in range(100):
  print(f"\nstarting the {i}th epoch\n")
  api.start_game(practice=1,verbose=True)



starting the 0th epoch

Successfully start a new game! Game ID: be3445e85a30. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
  [Logic] Using BERTAgent (General Strategy)
Guessing letter: e
Sever response: {'game_id': 'be3445e85a30', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ e _ e '}
  [Logic] Using BERTAgent (General Strategy)
Guessing letter: r
Sever response: {'game_id': 'be3445e85a30', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ e _ e '}
  [Logic] Using BERTAgent (General Strategy)
Guessing letter: a
Sever response: {'game_id': 'be3445e85a30', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ a _ e _ e '}
  [Logic] Using BERTAgent (General Strategy)
Guessing letter: n
Sever response: {'game_id': 'be3445e85a30', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e n _ a _ e _ e '}
  [Logic] Using BERTAgent (General Strategy)
Guessing letter: t
Sever response: {'game_id': 'be3445e85a30', 'status': 'ongoing', 'tries_remains': 4, 'word

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-442-4133030712.py", line 3, in <cell line: 0>
    api.start_game(practice=1,verbose=True)
  File "/tmp/ipython-input-439-898547016.py", line 104, in start_game
    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-439-898547016.py", line 157, in request
    response = self.session.request(
               ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/requests/sessions.py", line 579, in request
    settings = self.merge_environment_settings(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/requests/sessions.py"

In [ ]:
total_practice_successes/total_practice_runs

0.5285761119003387

In [ ]:
total_practice_successes

5461

In [ ]:
total_practice_runs

10364

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs- 10364, (total_practice_successes - 5461) / (total_practice_runs - 10364)))

run 310 practice games out of an allotted 100,000. practice success rate so far = 0.545


In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.